<a href="https://colab.research.google.com/github/Prajna1999/eng-or-mt/blob/master/tokenizer_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers sentencepiece

In [2]:
!pip install --upgrade huggingface_hub


In [3]:
from huggingface_hub import login
login()

In [4]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.3", use_fast=False)
print(tokenizer.bos_token)

tokenizer_config.json:   0%|          | 0.00/141k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

<s>


In [110]:
import unicodedata
import regex
class TokenizerWrapper:
    def __init__(self, base_tokenizer,suffixes=None,force_suffix_split=False):
      self.tokenizer=base_tokenizer
      self.vocab=self.tokenizer.get_vocab()

      # known Odia suffixes. In future expand the list
      self.suffixes=suffixes or[  "ମାନେ", "ଗୁଡ଼ିକ", "ରେ", "କୁ", "ରୁ", "ଟି"]
      self.force_suffix_split=force_suffix_split

    def normalize_unicode(self, text):
      normalized_text=unicodedata.normalize("NFC", text)
      cleaned = normalized_text.replace('\u200d', '').replace('\u200c', '')
      return cleaned

    def is_known_token(self,token):
      # sentencepiece tokenizer sometimes have "__token" structure in their vocab
      return token in self.vocab or f"__{token}" in self.vocab

    def split_suffix(self, word, depth=0):
      indent = "  " * depth
      for suffix in sorted(self.suffixes, key=len, reverse=True):
        if word.endswith(suffix) and len(word)>len(suffix)+1:
          stem=word[:-len(suffix)]

          if self.force_suffix_split or self.is_known_token(stem) or self.is_known_token(suffix):
            print(f"{indent}📍 Split: '{word}' → '{stem}' + '{suffix}'")
            split_stem = self.split_suffix(stem, depth + 1)
            return f"{split_stem} {suffix}"
          else:
            print(f"{indent}⚠️  Not splitting: '{word}' → both parts unknown")
      print(f"{indent}✅ Final Stem: '{word}'")
      return word

    def preprocess_text(self,text):
      normalized=self.normalize_unicode(text)
      words=normalized.split()
      processed=[self.split_suffix(word, 0) for word in words]
      return " ".join(processed)

    def visualize_token_split(self,word):
      print("\n🧪 Original word:", word)
      # raw tokenization
      raw_tokens=self.tokenizer.tokenize(word)
      print(f"Raw Tokens ({len(raw_tokens)}):", raw_tokens)
      split_word=self.split_suffix(word)
      pre_tokens=self.tokenizer.tokenize(split_word)
      print(f"🔹 Preprocessed: {split_word}")
      print(f"✅ New tokens ({len(pre_tokens)}):", pre_tokens)
      saved=len(raw_tokens) - len(pre_tokens)
      print(f"Token count reduced by : {saved} tokens\n{'-'*50}")

    def encode(self, text, **kwargs):
      preprocessed=self.preprocess_text(text)
      encoded=self.tokenizer.encode(preprocessed, **kwargs)
      return encoded


In [117]:
wrapper=TokenizerWrapper(tokenizer,force_suffix_split=False)
words = ["ପିଲାମାନେରେ", "ଛେଳିମାନେ", "ପାଠଟି", "ବইଗୁଡ଼ିକୁ"]
# wrapper.visualize_token_split("ପିଲାମାନେରେ ଘାସକୁ ଖାଉଛନ୍ତି ।")
print(len(wrapper.encode("ପିଲାମାନେରେ ଘାସକୁ ଖାଉଛନ୍ତି ।")))

⚠️  Not splitting: 'ପିଲାମାନେରେ' → both parts unknown
✅ Final Stem: 'ପିଲାମାନେରେ'
⚠️  Not splitting: 'ଘାସକୁ' → both parts unknown
✅ Final Stem: 'ଘାସକୁ'
✅ Final Stem: 'ଖାଉଛନ୍ତି'
✅ Final Stem: '।'
75
